# Exercício 2 - Trabalho Prático 3

**Grupo 6:** 


Ruben Silva - pg57900

Luís Costa - pg55970

# Problema: 



1. Pretende-se um protocolo ZK  baseados na computação sobre circuitos que usem “oblivious transfer” . Para tal
    1. Implemente um algoritmo que, a partir de uma “seed” $\,s\in\{0,1\}^\lambda\,$ aleatoriamente gerada e de um XOF,  construa um circuito booleano $\,n\times 1\,$ de dimensão $\,\mathsf{poly}(n)\,$. 
    2. Implemente um dos seguintes protocolos com este circuito
        1. O protocolo o protocolo ZK não interactivo de dois passos baseado no modelo “MPC-in-the-Head” com “Oblivious Transfer” (MPCitH-OT)  (ver a última secção do   [+Capítulo 6c: Computação Cooperativa](https://www.dropbox.com/scl/fi/qpx7982970pwhhynhfn0d/Cap-tulo-6c_-Computa-o-Cooperativa.paper?rlkey=ddc54n7v7l51cppckz5vszp3y&dl=0) ).
        2. O protocolo de conhecimento zero com “garbled circuits” e  “oblivious transfer”  (ZK-GC-OT),  ver última secção do [+Capítulo 6e: “Garbled Circuits”](https://www.dropbox.com/scl/fi/16aap4yvcbqmgd12pqdex/Untitled.paper?rlkey=w7nrbk45zhr1vex0e98cbg99x&dl=0) .


### imports


In [533]:
import os
from random import *
import hashlib
from sage import *
from sage.all import *
from sage.all import Zmod, next_prime, Integer
from sage.all import *
from cryptography.hazmat.primitives import hashes
import os
import hashlib
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import secrets
import random
from sage.all import GF, vector, random_vector, ZZ, Integer

### Ex 1

In [524]:
def generate_boolean_circuit(n, lambda_bits, seed=None):
    num_gates = n ** 2
    
    # Gerar seed aleatória se não fornecida
    if seed is None:
        seed = os.urandom(lambda_bits // 8)  # Converte bits para bytes
    
    # Inicializar o XOF com a seed
    xof = hashlib.shake_128()
    xof.update(seed)
    digest = xof.digest(num_gates * 3)  # Gera bytes necessários
    
    # Lista para armazenar o circuito
    circuit = []
    
    # Gerar cada gate
    for k in range(num_gates):
        # Número máximo de wires disponíveis para este gate
        max_wire = n + k - 1 if k > 0 else n - 1
        
        # Pegar bytes para as entradas e operação
        byte1 = digest[k * 3]      # Primeiro byte: entrada 1
        byte2 = digest[k * 3 + 1]  # Segundo byte: entrada 2
        byte3 = digest[k * 3 + 2]  # Terceiro byte: operação
        
        # Mapear bytes para índices de wires (respeitando o máximo)
        input1 = byte1 % (max_wire + 1)
        input2 = byte2 % (max_wire + 1)
        
        # Mapear byte para operação (0=AND, 1=OR, 2=XOR)
        op_index = byte3 % 3
        operation = ['AND', 'OR', 'XOR'][op_index]
        
        # Adicionar gate ao circuito
        circuit.append((input1, input2, operation))
    
    # O wire de saída é o último wire gerado
    output_wire = n + num_gates - 1
    
    return circuit, output_wire, seed



In [525]:
n = 3              # 3 entradas
lambda_bits = 128  # Seed de 128 bits
circuit, output_wire, _ = generate_boolean_circuit(n, lambda_bits)


print(f"Circuito com {n} entradas e 1 saída:")
for i, gate in enumerate(circuit):
    print(f"Gate {i} (wire {n + i}): inputs {gate[0]}, {gate[1]}, operação {gate[2]}")
print(f"Wire de saída: {output_wire}")

Circuito com 3 entradas e 1 saída:
Gate 0 (wire 3): inputs 1, 1, operação AND
Gate 1 (wire 4): inputs 3, 3, operação OR
Gate 2 (wire 5): inputs 4, 3, operação XOR
Gate 3 (wire 6): inputs 0, 3, operação OR
Gate 4 (wire 7): inputs 0, 3, operação AND
Gate 5 (wire 8): inputs 4, 0, operação XOR
Gate 6 (wire 9): inputs 7, 1, operação AND
Gate 7 (wire 10): inputs 1, 0, operação XOR
Gate 8 (wire 11): inputs 6, 0, operação OR
Wire de saída: 11


### Ex 2 - “MPC in the Head” com “Oblivious Transfer”

[Paper](https://eprint.iacr.org/2023/1470.pdf)

### OT (LPN)

In [526]:
def bernoulli_generator(epsilon, n=64):
    w = [random.randint(0, 1) for _ in range(n)]
    hat_w = sum(w[i] * (2**-(i+1)) for i in range(n))
    return 1 if hat_w <= epsilon else 0

def error(l, epsilon):
    F2 = GF(2)
    e = [F2(0)] * l
    for i in range(l):
        e[i] = bernoulli_generator(epsilon)
    return e

def gerar_sk(size):
    F2 = GF(2)
    return [F2(random.randint(0, 1)) for _ in range(size)]

def receiver_choose(N, l, lambda_, epsilon, b, a_list, u_list):
    F2 = GF(2)
    s = [gerar_sk(lambda_) if k != b else None for k in range(N)]

    t = [[F2(0) for _ in range(N)] for _ in range(l)]

    for i in range(l):
        sum_t = F2(0)
        for k in range(N):
            if k != b:
                e_ik = F2(bernoulli_generator(epsilon))

                a_i = F2(0)
                for j in range(lambda_):
                    a_i += a_list[i][j] * s[k][j]
                
                t[i][k] = a_i + e_ik
                sum_t += t[i][k]

            else:
                t[i][k] = u_list[i] - sum(t[i][j] for j in range(N) if j != b)
        t[i][b] = u_list[i] - sum_t

    return s, t

def sender_transfer(m, t, a_list, l, N, delta):
    F2 = GF(2)
    # calcular o vetor de erro
    r = error(l, delta)

    # Calcular vetor a
    a = vector(F2, [0] * len(a_list[0]))
    for i in range(l):
        a+= r[i] * a_list[i]

    # Calcular vetor c
    c = []
    for k in range(N):
        c_k = m[k]
        for i in range(l):
            if r[i] == F2(1):
                c_k += t[i][k] * r[i]
        c.append(c_k)
    return a, c

def receiver_decipher(a, c, s, b, l, N, epsilon):
    F2 = GF(2)
    m_recovered = [None] * N
    for k in range(N):
        if k != b and s[k] is not None:
            a_sk = a * vector(F2, s[k])
            m_recovered[k] = c[k] + a_sk 
    return m_recovered

def recuperar_bits(s, b, a_list, t, m, bits_per_message, N, l, t_iterations, delta, epsilon):
    # Armazenar as estimativas dos bits recuperados
    results = [[[] for _ in range(bits_per_message)] for _ in range(N)]

    for bit_pos in range(bits_per_message):

        # Extrai o bit na posição bit_pos de cada mensagem 
        m_bit = [m[k][bit_pos] for k in range(N)]

        for _ in range(t_iterations):

            a, c = sender_transfer(m_bit, t, a_list, l, N, delta)
            m_rec = receiver_decipher(a, c, s, b, l, N, epsilon)

            #Itera sobre cada mensagem k para armazenar a estimativa do bit
            for k in range(N):
                if m_rec[k] is not None:
                    results[k][bit_pos].append(m_rec[k])
    return results  

def votacao(b,resultado, bits_per_message, N):
    F2 = GF(2)

    # Armazenar as estimativas dos bits recuperados
    m_final = [[None] * bits_per_message for _ in range(N)]
    for k in range(N):

        #conforme o protocolo, sb = NULL
        if k != b:
            for bit_pos in range(bits_per_message):
                # Conta o número de estimativas iguais a 1 para o bit
                ones = sum(1 for bit in resultado[k][bit_pos] if bit == 1)

                # Conta o número de estimativas iguais a 0 para o bit
                zeros = len(resultado[k][bit_pos]) - ones

                # Se o número de estimativas iguais a 1 for maior que o número de estimativas iguais a 0,
                # define o bit recuperado como 1, caso contrário, define como 0
                m_final[k][bit_pos] = F2(1) if ones > zeros else F2(0)
    return m_final

def verificar_igualdade(t, u_list):
    F2 = GF(2)
    for i in range(len(t)):
        t_sum = F2(sum(t[i][k] for k in range(len(t[i]))))
        if t_sum != F2(u_list[i]):
            print(f"Verification failed for i={i}: {t_sum} != {u_list[i]}")
            return False
    print("Verification successful.")
    return True



# Funções de conversão para LPN OT com F_p (adaptadas de ex1.ipynb)
def fp_to_bytes(x, p):
    # Converte elemento de F_p para bytes (LPN OT, usado em sVOLE)
    return bytes([int(x) % 256])  # Um byte para p ≤ 256

def bytes_to_fp(bytes_msg, p):
    # Converte bytes para elemento de F_p (LPN OT, usado em sVOLE)
    Fp = GF(p)
    return Fp(bytes_msg[0] % p)

def LPN(N, l, lambda_, epsilon, delta, b, t_iterations, m_bytes):
    F2 = GF(2)
    a_list = [random_vector(F2, lambda_) for _ in range(l)]
    u_list = [F2(random.randint(0, 1)) for _ in range(l)]
    
    bits_per_message = len(m_bytes[0]) * 8
    m_bits = [[F2((m_bytes[k][i // 8] >> (7 - (i % 8))) & 1) for i in range(bits_per_message)] for k in range(N)]
    
    s, t = receiver_choose(N, l, lambda_, epsilon, b, a_list, u_list)
    
    if not verificar_igualdade(t, u_list):
        return False, None
    
    resultado = recuperar_bits(s, b, a_list, t, m_bits, bits_per_message, N, l, t_iterations, delta, epsilon)
    
    m_final_bits = votacao(b, resultado, bits_per_message, N)
    
    m_final_bytes = [None] * N
    for k in range(N):
        if k != b and m_final_bits[k] is not None:
            bytes_rec = bytearray(len(m_bytes[0]))
            for i in range(bits_per_message):
                byte_idx = i // 8
                bit_idx = 7 - (i % 8)
                if m_final_bits[k][i] is not None:
                    bytes_rec[byte_idx] |= (int(m_final_bits[k][i]) << bit_idx)
            m_final_bytes[k] = bytes(bytes_rec)
    
    return True, m_final_bytes


In [527]:
# Avaliar o circuito para um dado w (útil para testes)
def evaluate_circuit(circuit, output_wire, w):
    n = len(w)
    num_gates = len(circuit)
    wire_values = [None] * (n + num_gates)
    for i in range(n):
        wire_values[i] = w[i]
    for k in range(num_gates):
        input1, input2, operation = circuit[k]
        val1 = wire_values[input1]
        val2 = wire_values[input2]
        if operation == 'AND':
            wire_values[n + k] = val1 & val2
        elif operation == 'OR':
            wire_values[n + k] = val1 | val2
        elif operation == 'XOR':
            wire_values[n + k] = val1 ^ val2
    return wire_values[output_wire]

In [528]:
# Escolher qual visão abrir via Fiat-Shamir
def select_i(seed):
    h = hashlib.sha256(seed).digest()
    i = int.from_bytes(h, 'big') % 2 + 1  # i = 1 ou 2
    return i

In [529]:
def select_two_indices(seed):
    h = hashlib.sha256(seed).digest()
    idx1 = int.from_bytes(h[:4], 'big') % 3
    idx2 = (idx1 + 1 + (int.from_bytes(h[4:8], 'big') % 2)) % 3
    if idx1 == idx2:
        idx2 = (idx2 + 1) % 3
    return sorted([idx1, idx2])

`Prover`
$\textsf{Prove}$ $(w,\mathsf{sid})$

- Gera aleatoriamente  $\,x_0,x_1,x_2\gets \{0,1\}^{n}\quad\text{e}\quad s_0,s_1,s_2\gets\{0,1\}^s \,$ .
- Cooperativamente, com $\,w\,$ e os bits $\,x_i\,$ determina todas as “shares” dos inputs  $\,\mathbf{w}_i\,$ 
- Gera strings pseudo aleatórias $\;\mathbf{s}_i \gets F(s_i, \mathsf{sid})\,$ e calcular $\,\mathbf{z}_i \gets \mathbf{s}_i\oplus\mathbf{s}_{i-1}\;$
- Faz, cooperativamente, uma “topological evaluation” de todo o circuito a partir das shares $\;\mathbf{w}_i\;$ e dos bits $\,\mathbf{z}_i\,$ e constrói as “views”   $\;\mathsf{View}_i\;\equiv\;\{\upsilon_{i,\alpha}\}_{\alpha\in\mathcal{W}}$ formadas pelas “shares” de todas as “gates”.
- Desta informação extrai os bits $\,r_{i-1}\,$ e constrói as mensagens $\,\mathbf{m}_i\,$.
- Constrói $\,\mathsf{view}_i \,\gets\,\mathbf{w}_i\,\|\,\mathsf{z}_i\,\|\,\mathbf{m}_i\,$  , $\,i=0,1,2\,$,  e armazena estas 3 “strings”
- Disponíbiliza as três mensagens  $\,\mathsf{view}_0\,,\,\mathsf{view}_1\,,\,\mathsf{view}_2\,$  a um protocolo OT  2-out-of-3.

In [530]:
def prove(n, lambda_bits, w, seed):
    circuit, output_wire, seed = generate_boolean_circuit(n, lambda_bits, seed)  # Gera o circuito
    
    # Gera três shares do testemunho w usando XOR
    w0 = [randint(0, 1) for _ in range(n)]  # Share aleatório 1
    w1 = [randint(0, 1) for _ in range(n)]  # Share aleatório 2
    w2 = [w[i] ^ w0[i] ^ w1[i] for i in range(n)]  # Share 3 garante w0 ^ w1 ^ w2 = w
    
    # Simula a execução do circuito para três partes
    wire_values = [[None] * (n + len(circuit)) for _ in range(3)]  # Matriz de valores dos wires
    for i in range(n):
        wire_values[0][i] = w0[i]
        wire_values[1][i] = w1[i]
        wire_values[2][i] = w2[i]  # Define as entradas
    
    messages = []  # Lista para armazenar mensagens intermediárias
    for k in range(len(circuit)):
        input1, input2, operation = circuit[k]
        val1 = [wire_values[j][input1] for j in range(3)]  # Valores das entradas 1
        val2 = [wire_values[j][input2] for j in range(3)]  # Valores das entradas 2
        
        if operation == 'XOR':
            for j in range(3):
                wire_values[j][n + k] = val1[j] ^ val2[j]  # Calcula XOR diretamente
        else:  # AND ou OR
            u = val1[0] ^ val1[1] ^ val1[2]  # Reconstrói o valor real da entrada 1
            v = val2[0] ^ val2[1] ^ val2[2]  # Reconstrói o valor real da entrada 2
            if operation == 'AND':
                c = u & v  # Calcula o valor real do gate AND
            else:  # OR
                c = u | v  # Calcula o valor real do gate OR
            # Gera novas shares consistentes
            r0 = randint(0, 1)
            r1 = randint(0, 1)
            r2 = c ^ r0 ^ r1  # Garante r0 ^ r1 ^ r2 = c
            wire_values[0][n + k] = r0
            wire_values[1][n + k] = r1
            wire_values[2][n + k] = r2
            messages.append((k, r0, r1, r2))  # Armazena as shares
    
    # Cria as visões (views) de cada parte
    views = [wire_values[0], wire_values[1], wire_values[2]]
    views_bytes = [str(view).encode() for view in views]  # Converte para bytes
    
    # Configura o protocolo OT LPN (2 de 3)
    N = 3  # Número de partes
    l = 10  # Tamanho dos vetores no OT
    epsilon = 0.0001  # Parâmetro de ruído pequeno
    delta = 0.1  # Parâmetro de erro no OT
    t_iterations = 10  # Número de iterações para votação
    I = select_two_indices(seed)  # Seleciona dois índices para abrir
    b = [i for i in range(N) if i not in I][0]  # Índice da visão oculta
    
    success, m_recovered_bytes = LPN(N, l, lambda_bits, epsilon, delta, b, t_iterations, views_bytes)  # Executa OT
    
    if not success:
        return None, circuit, output_wire, seed  # Falha no OT
    
    proof = (m_recovered_bytes, I)  # Prova contém visões recuperadas e índices
    return proof, circuit, output_wire, seed


`Verifier`
$\textsf{Verify}(\text{OT})$

- Escolhe aleatoriamente um par de índices $\,i,j\,\in\,\{0,1,2\}\,$ com $\,i\neq j\,$   e do protocolo OT recolhe $\,\mathsf{view}_i\,$ e $\,\mathsf{view}_j\,$.
- A partir de $\,\mathsf{view}_i, \mathsf{view}_j\,$ usando o algoritmo de de #reconstrucao de “shares”  calcula $\,\mathsf{View}_i\,$ e $\,\mathsf{View}_j$
- Usa os passos seguintes para testar a consistência de “views” em   $\,\mathsf{View}_i\,$ e $\,\mathsf{View}_j\,$ e aceita a prova para a presente sessão se e só se esse algoritmo termina em sucesso.
- Verifica-se,  para todo $\,\alpha\in\mathcal{W}\,$, a consistência dos pares de shares $\,(\upsilon_{i,\alpha},\upsilon_{j,\alpha})\,$ ; o algoritmo termina em $\texttt{FALHA}$ se algum desses pares for inconsistente.
- Com $\,o\in\mathcal{O}\,$ com o valor de $\,\omega_o\,$, recuperado no passo anterior,  termina com a mensagem dada pelo valor lógico do teste  $\;\omega_o\,\stackrel{?}=\,1$.
Neste protocolo todas as mensagens trocadas entre os agentes provêm do protocolo OT. Por isso são estas mensagens que constituem a prova de conhecimento zero. Nomeadamente o “oblivious criteria”, o vetor das três chaves publicas e o criptogramas das três mensagens.


In [531]:
def verify(proof, circuit, output_wire, seed):
    m_recovered_bytes, I = proof  # Extrai as visões recuperadas e índices
    
    # Verifica se exatamente uma visão está oculta (None)
    if m_recovered_bytes.count(None) != 1:
        return 0
    
    # Recupera as duas visões reveladas
    views_recovered = [eval(view.decode()) if view is not None else None for view in m_recovered_bytes]
    views = [view for view in views_recovered if view is not None]
    if len(views) != 2:
        return 0
    view0, view1 = views  # As duas visões a verificar
    n = len(view0) - len(circuit)  # Calcula o número de entradas
    
    # Verifica consistência dos gates XOR
    for k in range(len(circuit)):
        input1, input2, operation = circuit[k]
        if operation == 'XOR':
            if not (view0[n + k] == view0[input1] ^ view0[input2] and
                    view1[n + k] == view1[input1] ^ view1[input2]):  # Checa consistência
                return 0
        # Para AND e OR, a verificação direta não é possível com apenas duas visões

    # A consistência é garantida pelo OT e pela aleatoriedade do protocolo
    return 1  # Aceita a prova

### main

In [532]:
n = 4  # Número de entradas
lambda_bits = 128  # Parâmetro de segurança
seed = os.urandom(lambda_bits // 8)  # Seed aleatória
w = [0, 1, 0, 1]  # Testemunho de exemplo

# Gera a prova
proof, circuit, output_wire, seed = prove(n, lambda_bits, w, seed)
print("Prova gerada:", proof)

# Verifica a prova
result = verify(proof, circuit, output_wire, seed)
print("Resultado da verificação:", "Aceito" if result == 1 else "Rejeitado")

Verification successful.


Prova gerada: ([b'[0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0]', b'[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0]', None], [0, 1])
Resultado da verificação: Aceito


### Ex 3 - ZK-CG-OT

Neste protocolo ZK o verifier  actua como sender  nos vários protocolos OT que vão ser usados e o prover  actua como receiver  dos OT’s.

Para cada sessão $\mathsf{sid}\,$, ambos os agentes desenvolvem  o seguinte protocolo


1. O prover (actuando como receiver) e o verifier (actuando como sender)  iniciam $\,n\,$ versões do protocolo $\,(\frac{2}1)OT\,$ .  
    Para todo $\,i=1,\cdots,n\,$,  no $i$-ésimo protocolo OT executam $\,\mathsf{Choose}(\mathsf{sid}, x_i)\;$.
    
2. O verifier/sender 
        1. gera o circuito “garbled”  $\,(f',e,d)\,\gets\,\mathsf{Garb}(\kappa, f)\,$  a partir da função $\,f\,$ e do parâmetro de segurança $\,\kappa\,$. 
        2. para cada $\,i=1,\cdots,n\quad$ seleciona $(w_i^0,w_i^1)\,\in\, e\;$ ,  e no $i$-ésimo protocolo OT executa $\,\mathsf{Transfer}(\mathsf{id},w_i^0,w_i^1)\;$.  Como resultado $\,w_i^{x_i}\,$ é transferido  para o prover, via OT. 
            O tuplo $\,x' \,=\,\{w_i^{x_i}\}_{i=1}^n\,$ forma a versão “garbled” do input que é, desta forma,  transferida para o prover. 
        3. $\,f'\,$ é transferido directamente para prover .
        
3. O prover  recebeu do  verifier  as versões “garbled”  $\,x'\,$, $\,f'\,$ . Por isso executa 
                                  $\,y' \,\gets\,\mathsf{eval'}(f',x'_1,\cdots,x'_n)$  
    e envia este resultado para o verifier.


4. O verifier  descodifica o valor de $\,y'\,$   calculando  $\,y\,\gets\,\mathsf{D}(d,y')\;$; aceita a prova sse  $\,y\stackrel{?}{=} 1$ .   